# Feature Creation Methods
Call these methods to add features to the Flights Training dataframe

In [6]:
from datetime import datetime
import pandas as pd
import pickle

In [7]:
filename = 'flight_2019'

infile = open(filename,'rb')
flight_2019 = pickle.load(infile)
infile.close()

#### New dataframe with a smaller sample of 500,000 records

In [8]:
flight_sample = flight_2019.sample(n=1000000, random_state=1)

In [9]:
flight_sample.shape

(1000000, 38)

## 1. Is Flight Delayed Feature (0 or 1 values)

In [10]:
def is_delayed_feature(flights_df):
    flights_df_cp = flights_df.copy()
    
    flights_df_cp['dep_delay'] = flights_df_cp['dep_delay'].fillna(0)
    flights_df_cp['arr_delay'] = flights_df_cp['arr_delay'].fillna(0)
    flights_df_cp['carrier_delay'] = flights_df_cp['carrier_delay'].fillna(0)
    flights_df_cp['weather_delay'] = flights_df_cp['weather_delay'].fillna(0)
    flights_df_cp['nas_delay'] = flights_df_cp['nas_delay'].fillna(0)
    flights_df_cp['security_delay'] = flights_df_cp['security_delay'].fillna(0)
    flights_df_cp['late_aircraft_delay'] = flights_df_cp['late_aircraft_delay'].fillna(0)

    flights_df_cp['total_delay'] = flights_df_cp.apply(
        lambda x: ((x['dep_delay'] + x['arr_delay'] +
                 x['carrier_delay'] +
                 x['weather_delay'] + x['nas_delay'] + x['security_delay'] +
                 x['late_aircraft_delay']
        )),  axis=1)
    
    return flights_df_cp.apply(
        lambda x: (0 if x['total_delay'] <= 0 else 1),  axis=1)
    

In [11]:
flight_sample['is_delayed'] = is_delayed_feature(flight_sample)
flight_sample.head()

,mkt_unique_carrier,branded_code_share,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,dest_airport_id,crs_dep_time,dep_time,...,nas_delay,security_delay,late_aircraft_delay,first_dep_time,total_add_gtime,longest_add_gtime,fl_date,year,month,is_delayed
5817519,DL,DL_CODESHARE,4016,OO,N291SY,4016,12953,13342,1747,1944.0,...,3.0,0.0,108.0,None,NaN,NaN,2019-05-23,2019.0,5.0,1
3440584,DL,DL_CODESHARE,4566,OO,N477CA,4566,10397,12007,2019,2017.0,...,NaN,NaN,NaN,None,NaN,NaN,2019-12-05,2019.0,12.0,0
2213756,DL,DL,2151,DL,N844DN,2151,10721,13487,1820,1802.0,...,NaN,NaN,NaN,None,NaN,NaN,2019-02-06,2019.0,2.0,0
4372882,AA,AA,439,AA,N155NN,439,13930,14107,705,656.0,...,NaN,NaN,NaN,None,NaN,NaN,2019-09-17,2019.0,9.0,0
2224046,NK,NK,318,NK,N907NK,318,14635,13487,1315,1311.0,...,NaN,NaN,NaN,None,NaN,NaN,2019-02-06,2019.0,2.0,0


## 2. Flight Speed

In [12]:
def flight_speed_feature(flights_df):
    flights_df_cp = flights_df.copy()

    if ("cancelled" in flights_df):
        flights_df_cp['distance'] = flights_df_cp.apply(
            lambda x: (0.0 if ((x['cancelled'] == 1) | (x['diverted'] == 1)) else x['distance']
                  ),  axis=1)
    
        flights_df_cp['crs_elapsed_time'] = flights_df_cp.apply(
            lambda x: (60.0 if ((x['cancelled'] == 1) | (x['diverted'] == 1)) else x['crs_elapsed_time']
                  ),  axis=1)
    
    return flights_df_cp.apply(
        lambda x: ((x['distance'] / (x['crs_elapsed_time']/60)
              )),  axis=1)

In [13]:
flight_sample['flight_speed'] = flight_speed_feature(flight_sample)
flight_sample.head()

,mkt_unique_carrier,branded_code_share,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,dest_airport_id,crs_dep_time,dep_time,...,security_delay,late_aircraft_delay,first_dep_time,total_add_gtime,longest_add_gtime,fl_date,year,month,is_delayed,flight_speed
5817519,DL,DL_CODESHARE,4016,OO,N291SY,4016,12953,13342,1747,1944.0,...,0.0,108.0,None,NaN,NaN,2019-05-23,2019.0,5.0,1,303.287671
3440584,DL,DL_CODESHARE,4566,OO,N477CA,4566,10397,12007,2019,2017.0,...,NaN,NaN,None,NaN,NaN,2019-12-05,2019.0,12.0,0,192.800000
2213756,DL,DL,2151,DL,N844DN,2151,10721,13487,1820,1802.0,...,NaN,NaN,None,NaN,NaN,2019-02-06,2019.0,2.0,0,309.357798
4372882,AA,AA,439,AA,N155NN,439,13930,14107,705,656.0,...,NaN,NaN,None,NaN,NaN,2019-09-17,2019.0,9.0,0,378.947368
2224046,NK,NK,318,NK,N907NK,318,14635,13487,1315,1311.0,...,NaN,NaN,None,NaN,NaN,2019-02-06,2019.0,2.0,0,369.391304


## 3. Flight Haul Type (values of 0, 1 and 2 for Short, Medium and Long)

In [14]:
def flight_haul_type_feature(flights_df):
    flights_df_cp = flights_df.copy()
    
    if ("cancelled" in flights_df):
        flights_df_cp['crs_elapsed_time'] = flights_df_cp.apply(
            lambda x: (0 if ((x['cancelled'] == 1) | (x['diverted'] == 1)) else x['crs_elapsed_time']
                  ),  axis=1)
    
    return flights_df_cp.apply(
        lambda x: 0 if x['crs_elapsed_time'] <= 180.0 else (2 if x['crs_elapsed_time'] >= 360.0 else 1), axis=1
)

In [15]:
flight_sample['flight_haul_type'] = flight_haul_type_feature(flight_sample)
flight_sample.head()

,mkt_unique_carrier,branded_code_share,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,dest_airport_id,crs_dep_time,dep_time,...,late_aircraft_delay,first_dep_time,total_add_gtime,longest_add_gtime,fl_date,year,month,is_delayed,flight_speed,flight_haul_type
5817519,DL,DL_CODESHARE,4016,OO,N291SY,4016,12953,13342,1747,1944.0,...,108.0,None,NaN,NaN,2019-05-23,2019.0,5.0,1,303.287671,0
3440584,DL,DL_CODESHARE,4566,OO,N477CA,4566,10397,12007,2019,2017.0,...,NaN,None,NaN,NaN,2019-12-05,2019.0,12.0,0,192.800000,0
2213756,DL,DL,2151,DL,N844DN,2151,10721,13487,1820,1802.0,...,NaN,None,NaN,NaN,2019-02-06,2019.0,2.0,0,309.357798,1
4372882,AA,AA,439,AA,N155NN,439,13930,14107,705,656.0,...,NaN,None,NaN,NaN,2019-09-17,2019.0,9.0,0,378.947368,1
2224046,NK,NK,318,NK,N907NK,318,14635,13487,1315,1311.0,...,NaN,None,NaN,NaN,2019-02-06,2019.0,2.0,0,369.391304,1


## 4. Month of the Year (1 to 12)

In [16]:
def month_feature(flights_df):
    flights_df_cp = flights_df.copy()
    
    return flights_df_cp[flights_df_cp['fl_date'].notna()].apply(
        lambda x: datetime.strptime(x['fl_date'], '%Y-%m-%d').date().month, axis=1)

In [17]:
flight_sample['flight_month'] = month_feature(flight_sample)
flight_sample.head()

,mkt_unique_carrier,branded_code_share,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,dest_airport_id,crs_dep_time,dep_time,...,first_dep_time,total_add_gtime,longest_add_gtime,fl_date,year,month,is_delayed,flight_speed,flight_haul_type,flight_month
5817519,DL,DL_CODESHARE,4016,OO,N291SY,4016,12953,13342,1747,1944.0,...,None,NaN,NaN,2019-05-23,2019.0,5.0,1,303.287671,0,5
3440584,DL,DL_CODESHARE,4566,OO,N477CA,4566,10397,12007,2019,2017.0,...,None,NaN,NaN,2019-12-05,2019.0,12.0,0,192.800000,0,12
2213756,DL,DL,2151,DL,N844DN,2151,10721,13487,1820,1802.0,...,None,NaN,NaN,2019-02-06,2019.0,2.0,0,309.357798,1,2
4372882,AA,AA,439,AA,N155NN,439,13930,14107,705,656.0,...,None,NaN,NaN,2019-09-17,2019.0,9.0,0,378.947368,1,9
2224046,NK,NK,318,NK,N907NK,318,14635,13487,1315,1311.0,...,None,NaN,NaN,2019-02-06,2019.0,2.0,0,369.391304,1,2


## 5. Arrival Hour of the Day (0 to 23)

In [18]:
def arrival_hour_of_day_feature(flights_df):
    flights_df_cp = flights_df.copy()
    
    return flights_df_cp[flights_df_cp['crs_arr_time'].notna()].apply(
        lambda x: 0 if x['crs_arr_time'] == 2400.0 else datetime.strptime(str(int(x['crs_arr_time'])).zfill(4), '%H%M').time().hour, axis=1)
    

In [19]:
flight_sample['arrival_hour_of_day'] = arrival_hour_of_day_feature(flight_sample)
flight_sample.head()

,mkt_unique_carrier,branded_code_share,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,dest_airport_id,crs_dep_time,dep_time,...,total_add_gtime,longest_add_gtime,fl_date,year,month,is_delayed,flight_speed,flight_haul_type,flight_month,arrival_hour_of_day
5817519,DL,DL_CODESHARE,4016,OO,N291SY,4016,12953,13342,1747,1944.0,...,NaN,NaN,2019-05-23,2019.0,5.0,1,303.287671,0,5,19
3440584,DL,DL_CODESHARE,4566,OO,N477CA,4566,10397,12007,2019,2017.0,...,NaN,NaN,2019-12-05,2019.0,12.0,0,192.800000,0,12,20
2213756,DL,DL,2151,DL,N844DN,2151,10721,13487,1820,1802.0,...,NaN,NaN,2019-02-06,2019.0,2.0,0,309.357798,1,2,20
4372882,AA,AA,439,AA,N155NN,439,13930,14107,705,656.0,...,NaN,NaN,2019-09-17,2019.0,9.0,0,378.947368,1,9,8
2224046,NK,NK,318,NK,N907NK,318,14635,13487,1315,1311.0,...,NaN,NaN,2019-02-06,2019.0,2.0,0,369.391304,1,2,16


## 6. Departure Hour of the Day (0 to 23)

In [20]:
def departure_hour_of_day_feature(flights_df):
    flights_df_cp = flights_df.copy()
    
    return flights_df_cp[flights_df_cp['crs_dep_time'].notna()].apply(
        lambda x: 0 if x['crs_dep_time'] == 2400.0 else datetime.strptime(str(int(x['crs_dep_time'])).zfill(4), '%H%M').time().hour, axis=1)
  

In [21]:
flight_sample['departure_hour_of_day'] = departure_hour_of_day_feature(flight_sample)
flight_sample.head()

,mkt_unique_carrier,branded_code_share,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,dest_airport_id,crs_dep_time,dep_time,...,longest_add_gtime,fl_date,year,month,is_delayed,flight_speed,flight_haul_type,flight_month,arrival_hour_of_day,departure_hour_of_day
5817519,DL,DL_CODESHARE,4016,OO,N291SY,4016,12953,13342,1747,1944.0,...,NaN,2019-05-23,2019.0,5.0,1,303.287671,0,5,19,17
3440584,DL,DL_CODESHARE,4566,OO,N477CA,4566,10397,12007,2019,2017.0,...,NaN,2019-12-05,2019.0,12.0,0,192.800000,0,12,20,20
2213756,DL,DL,2151,DL,N844DN,2151,10721,13487,1820,1802.0,...,NaN,2019-02-06,2019.0,2.0,0,309.357798,1,2,20,18
4372882,AA,AA,439,AA,N155NN,439,13930,14107,705,656.0,...,NaN,2019-09-17,2019.0,9.0,0,378.947368,1,9,8,7
2224046,NK,NK,318,NK,N907NK,318,14635,13487,1315,1311.0,...,NaN,2019-02-06,2019.0,2.0,0,369.391304,1,2,16,13


## 7. Percent of Flights Delay by Month

In [115]:
#flight_sample = flight_sample.drop(columns='arr_month')
#flight_sample = flight_sample.drop(columns=['percent_delay_x','percent_delay_y'])

In [22]:
def percent_delayed_by_month(flights_df):
    flights_df_cp = flights_df.copy()
    
    # make re-runnable
    if ('percent_delay' in flights_df_cp.columns):
        flights_df_cp = flights_df_cp.drop(columns=['percent_delay'])
        
    # Load the delay by month dataframe
    filename = './reference/percent_delay_by_month_2019'

    infile = open(filename,'rb')
    delay_df = pickle.load(infile)
    infile.close()
    
    # Join flights_df to delay_df, and return only the percent_delay column
    delay_join = pd.merge(flights_df_cp, 
                 delay_df,
                 how="inner",
                 left_on=['origin_airport_id','flight_month'],
                 right_on=['origin_airport_id','arr_month'])
    delay_join = delay_join.drop(columns='arr_month')
    return delay_join


In [23]:
flight_sample = percent_delayed_by_month(flight_sample)
flight_sample.head()

,mkt_unique_carrier,branded_code_share,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,dest_airport_id,crs_dep_time,dep_time,...,fl_date,year,month,is_delayed,flight_speed,flight_haul_type,flight_month,arrival_hour_of_day,departure_hour_of_day,percent_delay
0,DL,DL_CODESHARE,4016,OO,N291SY,4016,12953,13342,1747,1944.0,...,2019-05-23,2019.0,5.0,1,303.287671,0,5,19,17,0.00194
1,AA,AA_CODESHARE,4523,YX,N422YX,4523,12953,14730,900,855.0,...,2019-05-10,2019.0,5.0,0,270.821918,0,5,11,9,0.00194
2,DL,DL,311,DL,N952AT,311,12953,13930,1610,2034.0,...,2019-05-27,2019.0,5.0,1,249.886364,0,5,18,16,0.00194
3,WN,WN,2151,WN,N712SW,2151,12953,13232,1850,1846.0,...,2019-05-14,2019.0,5.0,0,290.000000,0,5,20,18,0.00194
4,DL,DL_CODESHARE,5353,9E,N349PQ,5353,12953,11057,1100,1131.0,...,2019-05-24,2019.0,5.0,1,234.820144,0,5,13,11,0.00194


## 8. Percent of Flights per Hour

In [24]:
def percent_flights_per_hour(flights_df):
    flights_df_cp = flights_df.copy()
    
    # make re-runnable
    if ('percent_flights' in flights_df_cp.columns):
        flights_df_cp = flights_df_cp.drop(columns=['percent_flights'])
    
    
    filename = './reference/percent_flights_by_hour_2019'

    infile = open(filename,'rb')
    local_df = pickle.load(infile)
    infile.close()
    
    local_join = pd.merge(flights_df_cp, 
                 local_df,
                 how="inner",
                 left_on=['origin_airport_id','arrival_hour_of_day'],
                 right_on=['origin_airport_id','hour_of_day'])
    
    local_join = local_join.drop(columns='hour_of_day')
    
    return local_join

In [25]:
flight_sample = percent_flights_per_hour(flight_sample)
flight_sample.head()

,mkt_unique_carrier,branded_code_share,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,dest_airport_id,crs_dep_time,dep_time,...,year,month,is_delayed,flight_speed,flight_haul_type,flight_month,arrival_hour_of_day,departure_hour_of_day,percent_delay,percent_flights
0,DL,DL_CODESHARE,4016,OO,N291SY,4016,12953,13342,1747,1944.0,...,2019.0,5.0,1,303.287671,0,5,19,17,0.00194,0.001349
1,UA,UA_CODESHARE,4235,EV,N14568,4235,12953,11042,1710,1858.0,...,2019.0,5.0,1,224.464286,0,5,19,17,0.00194,0.001349
2,AA,AA_CODESHARE,4330,YX,N105HQ,4330,12953,11278,1800,1814.0,...,2019.0,5.0,1,142.666667,0,5,19,18,0.00194,0.001349
3,AA,AA_CODESHARE,3473,MQ,N845AE,3473,12953,13244,1718,1723.0,...,2019.0,5.0,1,321.000000,0,5,19,17,0.00194,0.001349
4,UA,UA,1709,UA,N807UA,1709,12953,13930,1800,1849.0,...,2019.0,5.0,1,268.170732,0,5,19,18,0.00194,0.001349


## 9. Average Flights per Airport

In [26]:
def average_flights_per_airport(flights_df):
    flights_df_cp = flights_df.copy()
    
    # make re-runnable
    if ('avg_flights' in flights_df_cp.columns):
        flights_df_cp = flights_df_cp.drop(columns=['avg_flights'])
    
    filename = './reference/average_flights_by_airport_2019'

    infile = open(filename,'rb')
    local_df = pickle.load(infile)
    infile.close()
    
    local_join = pd.merge(flights_df_cp, 
                 local_df,
                 how="inner",
                 left_on=['origin_airport_id'],
                 right_on=['origin_airport_id'])
    return local_join

In [27]:
flight_sample = average_flights_per_airport(flight_sample)
flight_sample.head()

,mkt_unique_carrier,branded_code_share,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,dest_airport_id,crs_dep_time,dep_time,...,month,is_delayed,flight_speed,flight_haul_type,flight_month,arrival_hour_of_day,departure_hour_of_day,percent_delay,percent_flights,avg_flights
0,DL,DL_CODESHARE,4016,OO,N291SY,4016,12953,13342,1747,1944.0,...,5.0,1,303.287671,0,5,19,17,0.00194,0.001349,1772.5
1,UA,UA_CODESHARE,4235,EV,N14568,4235,12953,11042,1710,1858.0,...,5.0,1,224.464286,0,5,19,17,0.00194,0.001349,1772.5
2,AA,AA_CODESHARE,4330,YX,N105HQ,4330,12953,11278,1800,1814.0,...,5.0,1,142.666667,0,5,19,18,0.00194,0.001349,1772.5
3,AA,AA_CODESHARE,3473,MQ,N845AE,3473,12953,13244,1718,1723.0,...,5.0,1,321.000000,0,5,19,17,0.00194,0.001349,1772.5
4,UA,UA,1709,UA,N807UA,1709,12953,13930,1800,1849.0,...,5.0,1,268.170732,0,5,19,18,0.00194,0.001349,1772.5


## 10. Average Passengers By Month

In [28]:
def average_passengers_by_month(flights_df):
    flights_df_cp = flights_df.copy()
    
    # make re-runnable
    if ('avg_passengers' in flights_df_cp.columns):
        flights_df_cp = flights_df_cp.drop(columns=['avg_passengers'])
    
    filename = './reference/average_passengers_by_airport_2019'

    infile = open(filename,'rb')
    local_df = pickle.load(infile)
    infile.close()
    
    local_join = pd.merge(flights_df_cp, 
                 local_df,
                 how="inner",
                 left_on=['origin_airport_id'],
                 right_on=['origin_airport_id'])
    return local_join

In [29]:
flight_sample = average_passengers_by_month(flight_sample)
flight_sample.head()

,mkt_unique_carrier,branded_code_share,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,dest_airport_id,crs_dep_time,dep_time,...,is_delayed,flight_speed,flight_haul_type,flight_month,arrival_hour_of_day,departure_hour_of_day,percent_delay,percent_flights,avg_flights,avg_passengers
0,DL,DL_CODESHARE,4016,OO,N291SY,4016,12953,13342,1747,1944.0,...,1,303.287671,0,5,19,17,0.00194,0.001349,1772.5,342.0
1,UA,UA_CODESHARE,4235,EV,N14568,4235,12953,11042,1710,1858.0,...,1,224.464286,0,5,19,17,0.00194,0.001349,1772.5,342.0
2,AA,AA_CODESHARE,4330,YX,N105HQ,4330,12953,11278,1800,1814.0,...,1,142.666667,0,5,19,18,0.00194,0.001349,1772.5,342.0
3,AA,AA_CODESHARE,3473,MQ,N845AE,3473,12953,13244,1718,1723.0,...,1,321.000000,0,5,19,17,0.00194,0.001349,1772.5,342.0
4,UA,UA,1709,UA,N807UA,1709,12953,13930,1800,1849.0,...,1,268.170732,0,5,19,18,0.00194,0.001349,1772.5,342.0


## 11. Total Distance By Month

In [30]:
def total_distance_by_month(flights_df):
    flights_df_cp = flights_df.copy()
    
    # make re-runnable
    if ('total_distance' in flights_df_cp.columns):
        flights_df_cp = flights_df_cp.drop(columns=['total_distance'])
    
    
    filename = './reference/total_distance_per_month_by_airport_2019'

    infile = open(filename,'rb')
    local_df = pickle.load(infile)
    infile.close()
    
    local_join = pd.merge(flights_df_cp, 
                 local_df,
                 how="inner",
                 left_on=['origin_airport_id','flight_month'],
                 right_on=['origin_airport_id','arr_month'])
    
    local_join = local_join.drop(columns='arr_month')
    
    return local_join

In [31]:
flight_sample = total_distance_by_month(flight_sample)
flight_sample.head()

,mkt_unique_carrier,branded_code_share,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,dest_airport_id,crs_dep_time,dep_time,...,flight_speed,flight_haul_type,flight_month,arrival_hour_of_day,departure_hour_of_day,percent_delay,percent_flights,avg_flights,avg_passengers,total_distance
0,DL,DL_CODESHARE,4016,OO,N291SY,4016,12953,13342,1747,1944.0,...,303.287671,0,5,19,17,0.00194,0.001349,1772.5,342.0,1263586.0
1,UA,UA_CODESHARE,4235,EV,N14568,4235,12953,11042,1710,1858.0,...,224.464286,0,5,19,17,0.00194,0.001349,1772.5,342.0,1263586.0
2,AA,AA_CODESHARE,4330,YX,N105HQ,4330,12953,11278,1800,1814.0,...,142.666667,0,5,19,18,0.00194,0.001349,1772.5,342.0,1263586.0
3,AA,AA_CODESHARE,3473,MQ,N845AE,3473,12953,13244,1718,1723.0,...,321.000000,0,5,19,17,0.00194,0.001349,1772.5,342.0,1263586.0
4,UA,UA,1709,UA,N807UA,1709,12953,13930,1800,1849.0,...,268.170732,0,5,19,18,0.00194,0.001349,1772.5,342.0,1263586.0


## 12. Total Passengers By Month

In [ ]:
flight_sample = flight_sample.drop(columns=['month'])

In [41]:
def total_passengers_by_month(flights_df):
    flights_df_cp = flights_df.copy()
    
    # make re-runnable
    if ('total_passengers' in flights_df_cp.columns):
        flights_df_cp = flights_df_cp.drop(columns=['total_passengers'])
    
    
    filename = './reference/total_passengers_by_airport_2019'

    infile = open(filename,'rb')
    local_df = pickle.load(infile)
    infile.close()
    
    local_join = pd.merge(flights_df_cp, 
                 local_df,
                 how="inner",
                 left_on=['origin_airport_id','flight_month'],
                 right_on=['origin_airport_id','month'])
    #print(local_join.columns)
    local_join = local_join.drop(columns='month')
    
    return local_join

In [42]:
flight_sample = total_passengers_by_month(flight_sample)
flight_sample.head()

,mkt_unique_carrier,branded_code_share,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,dest_airport_id,crs_dep_time,dep_time,...,flight_haul_type,flight_month,arrival_hour_of_day,departure_hour_of_day,percent_delay,percent_flights,avg_flights,avg_passengers,total_distance,total_passengers
0,DL,DL_CODESHARE,4016,OO,N291SY,4016,12953,13342,1747,1944.0,...,0,5,19,17,0.00194,0.001349,1772.5,342.0,1263586.0,345
1,UA,UA_CODESHARE,4235,EV,N14568,4235,12953,11042,1710,1858.0,...,0,5,19,17,0.00194,0.001349,1772.5,342.0,1263586.0,345
2,AA,AA_CODESHARE,4330,YX,N105HQ,4330,12953,11278,1800,1814.0,...,0,5,19,18,0.00194,0.001349,1772.5,342.0,1263586.0,345
3,AA,AA_CODESHARE,3473,MQ,N845AE,3473,12953,13244,1718,1723.0,...,0,5,19,17,0.00194,0.001349,1772.5,342.0,1263586.0,345
4,UA,UA,1709,UA,N807UA,1709,12953,13930,1800,1849.0,...,0,5,19,18,0.00194,0.001349,1772.5,342.0,1263586.0,345


## 13. Average Fuel by Carrier

In [45]:
def average_fuel(flights_df):
    flights_df_cp = flights_df.copy()
    
    # make re-runnable
    if ('avg_fuel' in flights_df_cp.columns):
        flights_df_cp = flights_df_cp.drop(columns=['avg_fuel'])
    
    
    filename = './reference/average_fuel_by_carrier_2019'

    infile = open(filename,'rb')
    local_df = pickle.load(infile)
    infile.close()
    
    local_join = pd.merge(flights_df_cp, 
                 local_df,
                 how="inner",
                 left_on=['op_unique_carrier'],
                 right_on=['unique_carrier'])

    local_join = local_join.drop(columns='unique_carrier')
    
    return local_join

In [46]:
flight_sample = average_fuel(flight_sample)
flight_sample.head()

,mkt_unique_carrier,branded_code_share,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,dest_airport_id,crs_dep_time,dep_time,...,flight_month,arrival_hour_of_day,departure_hour_of_day,percent_delay,percent_flights,avg_flights,avg_passengers,total_distance,total_passengers,avg_fuel
0,DL,DL_CODESHARE,4016,OO,N291SY,4016,12953,13342,1747,1944.0,...,5,19,17,0.00194,0.001349,1772.5,342.0,1263586.0,345,8790766.0
1,DL,DL_CODESHARE,4016,OO,N292SY,4016,12953,13342,1747,1929.0,...,5,19,17,0.00194,0.001349,1772.5,342.0,1263586.0,345,8790766.0
2,DL,DL_CODESHARE,4016,OO,N278SY,4016,12953,13342,1747,1750.0,...,5,19,17,0.00194,0.001349,1772.5,342.0,1263586.0,345,8790766.0
3,DL,DL_CODESHARE,3616,OO,N268SY,3616,12953,11995,1729,1851.0,...,5,19,17,0.00194,0.001349,1772.5,342.0,1263586.0,345,8790766.0
4,DL,DL_CODESHARE,3607,OO,N276SY,3607,12953,11995,910,906.0,...,5,11,9,0.00194,0.001686,1772.5,342.0,1263586.0,345,8790766.0


In [47]:
flight_sample.shape

(952618, 50)

## 14. Add airport State

In [48]:
def origin_aiport_state(flights_df):
    flights_df_cp = flights_df.copy()
    
    # make re-runnable
    if ('origin_state' in flights_df_cp.columns):
        flights_df_cp = flights_df_cp.drop(columns=['origin_state'])
    
    
    filename = './reference/airport_state_mapping'

    infile = open(filename,'rb')
    local_df = pickle.load(infile)
    infile.close()
    
    local_join = pd.merge(flights_df_cp, 
                 local_df,
                 how="inner",
                 left_on=['origin_airport_id'],
                 right_on=['airport_id'])

    local_join = local_join.drop(columns='airport_id')
    
    return local_join

In [49]:
flight_sample = origin_aiport_state(flight_sample)
flight_sample.head()

,mkt_unique_carrier,branded_code_share,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,dest_airport_id,crs_dep_time,dep_time,...,arrival_hour_of_day,departure_hour_of_day,percent_delay,percent_flights,avg_flights,avg_passengers,total_distance,total_passengers,avg_fuel,state
0,DL,DL_CODESHARE,4016,OO,N291SY,4016,12953,13342,1747,1944.0,...,19,17,0.00194,0.001349,1772.5,342.0,1263586.0,345,8790766.0,NY
1,DL,DL_CODESHARE,4016,OO,N292SY,4016,12953,13342,1747,1929.0,...,19,17,0.00194,0.001349,1772.5,342.0,1263586.0,345,8790766.0,NY
2,DL,DL_CODESHARE,4016,OO,N278SY,4016,12953,13342,1747,1750.0,...,19,17,0.00194,0.001349,1772.5,342.0,1263586.0,345,8790766.0,NY
3,DL,DL_CODESHARE,3616,OO,N268SY,3616,12953,11995,1729,1851.0,...,19,17,0.00194,0.001349,1772.5,342.0,1263586.0,345,8790766.0,NY
4,DL,DL_CODESHARE,3607,OO,N276SY,3607,12953,11995,910,906.0,...,11,9,0.00194,0.001686,1772.5,342.0,1263586.0,345,8790766.0,NY


## 15. Weather by State and Month

In [50]:
def weather_by_state(flights_df):
    flights_df_cp = flights_df.copy()
    
    filename = './reference/weather_by_month'

    infile = open(filename,'rb')
    local_df = pickle.load(infile)
    infile.close()
    
    local_join = pd.merge(flights_df_cp, 
                 local_df,
                 how="inner",
                 left_on=['state','flight_month'],
                 right_on=['state','month'])

    local_join = local_join.drop(columns='month')
    
    return local_join

In [51]:
flight_sample = weather_by_state(flight_sample)
flight_sample.head()

      avgCloud avgDryDays avgRainDays avgSnowDays avgThunderDays  month state
0     71.88902          6          21           9              2      1    AK
1     59.98634          8          15          10              2      2    AK
2     63.13468          7          15          14              3      3    AK
3     65.64491          6          22           4              2      4    AK
4     59.64408          9          20           0              0      5    AK
..         ...        ...         ...         ...            ...    ...   ...
631  28.769823          4          26           0              4      8    WY
632  33.132973          7          21           0              2      9    WY
633   29.65525         13          15           0              1     10    WY
634  34.495274         15          11           0              0     11    WY
635  37.472916         17          12           0              1     12    WY

[636 rows x 7 columns]


,mkt_unique_carrier,branded_code_share,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,dest_airport_id,crs_dep_time,dep_time,...,avg_passengers,total_distance,total_passengers,avg_fuel,state,avgCloud,avgDryDays,avgRainDays,avgSnowDays,avgThunderDays
0,DL,DL_CODESHARE,4016,OO,N291SY,4016,12953,13342,1747,1944.0,...,342.0,1263586.0,345,8790766.0,NY,43.417507,12,17,0,3
1,DL,DL_CODESHARE,4016,OO,N292SY,4016,12953,13342,1747,1929.0,...,342.0,1263586.0,345,8790766.0,NY,43.417507,12,17,0,3
2,DL,DL_CODESHARE,4016,OO,N278SY,4016,12953,13342,1747,1750.0,...,342.0,1263586.0,345,8790766.0,NY,43.417507,12,17,0,3
3,DL,DL_CODESHARE,3616,OO,N268SY,3616,12953,11995,1729,1851.0,...,342.0,1263586.0,345,8790766.0,NY,43.417507,12,17,0,3
4,DL,DL_CODESHARE,3607,OO,N276SY,3607,12953,11995,910,906.0,...,342.0,1263586.0,345,8790766.0,NY,43.417507,12,17,0,3


## 16. Weather by Month

In [55]:
def weather_patterns(flights_df):
    flights_df_cp = flights_df.copy()
    
    weather_pattern_df = pd.read_csv('./reference/Flight_Pattern_Data.csv', index_col=0)
    weather_pattern_df

    
    local_join = pd.merge(flights_df_cp, 
                 weather_pattern_df,
                 how="inner",
                 left_on=['flight_month'],
                 right_on=['month'])

    local_join = local_join.drop(columns='month')
    
    return local_join



In [56]:
flight_sample = weather_patterns(flight_sample)
flight_sample.head()

,mkt_unique_carrier,branded_code_share,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,dest_airport_id,crs_dep_time,dep_time,...,early_count,early_avgDryDays,early_avgRainDays,early_avgCloud,early_avgSnowDays,delay_count,delay_avgDryDays,delay_avgRainDays,delay_avgCloud,delay_avgSnowDays
0,DL,DL_CODESHARE,4016,OO,N291SY,4016,12953,13342,1747,1944.0,...,55.5,17,11,3,0,30.8,18,10,3,0
1,DL,DL_CODESHARE,4016,OO,N292SY,4016,12953,13342,1747,1929.0,...,55.5,17,11,3,0,30.8,18,10,3,0
2,DL,DL_CODESHARE,4016,OO,N278SY,4016,12953,13342,1747,1750.0,...,55.5,17,11,3,0,30.8,18,10,3,0
3,DL,DL_CODESHARE,3616,OO,N268SY,3616,12953,11995,1729,1851.0,...,55.5,17,11,3,0,30.8,18,10,3,0
4,DL,DL_CODESHARE,3607,OO,N276SY,3607,12953,11995,910,906.0,...,55.5,17,11,3,0,30.8,18,10,3,0


# Export Sample Data

In [57]:
compression_opts = dict(method='zip',
                        archive_name='out.csv')  
flight_sample.to_csv('flight_sample1.zip', index=False,
          compression=compression_opts)  